## Smooth Landscape

In [13]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    x, y = np.asarray(x, dtype=float), np.asarray(y, dtype=float)
    return (x + 2*y - 7)**2 + (2*x + y - 5)**2


def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn, -10, 10, -10, 10, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Booth):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Booth):
Complete Graph: 1.586598
Hypercube Q8: 0.527433
Grid: 0.038429
Grid + Diagonals: 0.110331
Grid + Main Diagonals: 0.043096
QMOA Complete: 0.959174
QMOA Complete + local Diagonals: 1.447851
QMOA Complete + Principal Complete Diagonals: 1.777778
Grid + Complete Diagonals: 0.147765
Grid with Parallel Edges: 0.076859


In [15]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    return  0.5 * ((x**4 - 16 * x**2 + 5 * x) + (y**4 - 16 * y**2 + 5 * y))


def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn, -5, 5, -5, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Styblinski-Tang)):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Styblinski-Tang)):
Complete Graph: 3.116900
Hypercube Q8: 0.613389
Grid: 0.038429
Grid + Diagonals: 0.110331
Grid + Main Diagonals: 0.043096
QMOA Complete: 2.747651
QMOA Complete + local Diagonals: 2.725694
QMOA Complete + Principal Complete Diagonals: 0.273396
Grid + Complete Diagonals: 0.140459
Grid with Parallel Edges: 0.076859


In [14]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y,a=1,b=100):
    return (a - x)**2 + b * (y - x**2)**2


def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-5, 5, -5, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Rosenbrock)):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Rosenbrock)):
Complete Graph: 2.595910
Hypercube Q8: 0.039297
Grid: 0.038429
Grid + Diagonals: 0.110331
Grid + Main Diagonals: 0.043096
QMOA Complete: 0.855450
QMOA Complete + local Diagonals: 1.788046
QMOA Complete + Principal Complete Diagonals: 0.011327
Grid + Complete Diagonals: 0.022408
Grid with Parallel Edges: 0.076859


In [16]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    return 100 * np.sqrt(np.abs(y - 0.01 * x**2)) + 0.01 * np.abs(x + 10)


def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn, -15, -5, -3, 3, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Bukin N6)):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Bukin N6)):
Complete Graph: 5.020947
Hypercube Q8: 0.707079
Grid: 0.025868
Grid + Diagonals: 0.037186
Grid + Main Diagonals: 0.023925
QMOA Complete: 2.015986
QMOA Complete + local Diagonals: 2.253560
QMOA Complete + Principal Complete Diagonals: 2.086750
Grid + Complete Diagonals: 0.097180
Grid with Parallel Edges: 0.076859


In [17]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2


def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn, -5, 5, -5, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Himmelblau)):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Himmelblau)):
Complete Graph: 0.658377
Hypercube Q8: 0.001937
Grid: 0.000788
Grid + Diagonals: 0.003466
Grid + Main Diagonals: 0.001035
QMOA Complete: 0.216158
QMOA Complete + local Diagonals: 0.314174
QMOA Complete + Principal Complete Diagonals: 0.226937
Grid + Complete Diagonals: 0.004888
Grid with Parallel Edges: 0.001179


In [18]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    x, y = np.asarray(x, dtype=float), np.asarray(y, dtype=float)
    term1 = (1.5 - x + x*y)**2
    term2 = (2.25 - x + x*y**2)**2
    term3 = (2.625 - x + x*y**3)**2
    return term1 + term2 + term3


def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-4.5, 4.5, -4.5, 4.5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Beale)):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Beale)):
Complete Graph: 0.257496
Hypercube Q8: 0.210813
Grid: 0.038429
Grid + Diagonals: 0.110331
Grid + Main Diagonals: 0.043096
QMOA Complete: 0.257047
QMOA Complete + local Diagonals: 0.257082
QMOA Complete + Principal Complete Diagonals: 0.257047
Grid + Complete Diagonals: 0.147765
Grid with Parallel Edges: 0.076859


In [19]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    x, y = np.asarray(x, dtype=float), np.asarray(y, dtype=float)
    term1 = 1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)
    term2 = 30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2)
    return term1 * term2


def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-2, 2, -2, 2, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Goldstein-Price)):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Goldstein-Price)):
Complete Graph: 2.176246
Hypercube Q8: 0.007287
Grid: 0.038429
Grid + Diagonals: 0.110331
Grid + Main Diagonals: 0.043096
QMOA Complete: 1.733295
QMOA Complete + local Diagonals: 1.869968
QMOA Complete + Principal Complete Diagonals: 1.733431
Grid + Complete Diagonals: 0.147765
Grid with Parallel Edges: 0.076859


## Deceptive Landscape

In [11]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    a, b, c = 20, 0.2, 2 * np.pi
    x, y = np.array(x), np.array(y)
    sum_sq = 0.5 * (x ** 2 + y ** 2)
    cos_comp = 0.5 * (np.cos(c * x) + np.cos(c * y))
    return -a * np.exp(-b * np.sqrt(sum_sq)) - np.exp(cos_comp) + a + np.exp(1)



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-4.9, 5, -4.9, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Acklay)):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Acklay)):
Complete Graph: 0.314529
Hypercube Q8: 0.185427
Grid: 0.038429
Grid + Diagonals: 0.110331
Grid + Main Diagonals: 0.043096
QMOA Complete: 0.377826
QMOA Complete + local Diagonals: 0.349295
QMOA Complete + Principal Complete Diagonals: 0.186401
Grid + Complete Diagonals: 0.050331
Grid with Parallel Edges: 0.076859


In [21]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    return -(y + 47) * np.sin(np.sqrt(abs(x/2 + (y + 47)))) \
           - x * np.sin(np.sqrt(abs(x - (y + 47))))



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-5, 5, -5, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Eggholder)):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Eggholder)):
Complete Graph: 0.712926
Hypercube Q8: 0.689986
Grid: 0.038429
Grid + Diagonals: 0.110331
Grid + Main Diagonals: 0.043096
QMOA Complete: 0.697307
QMOA Complete + local Diagonals: 0.733412
QMOA Complete + Principal Complete Diagonals: 0.111916
Grid + Complete Diagonals: 0.095885
Grid with Parallel Edges: 0.076859


In [22]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    return 418.9829*2 - (x*np.sin(np.sqrt(np.abs(x))) + y*np.sin(np.sqrt(np.abs(y))))



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-5, 5, -5, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Schwefel)):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Schwefel)):
Complete Graph: 0.139478
Hypercube Q8: 0.153278
Grid: 0.038429
Grid + Diagonals: 0.110331
Grid + Main Diagonals: 0.043096
QMOA Complete: 0.153240
QMOA Complete + local Diagonals: 0.153211
QMOA Complete + Principal Complete Diagonals: 0.009148
Grid + Complete Diagonals: 0.008059
Grid with Parallel Edges: 0.076859


In [23]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    w1 = 1 + (x - 1) / 4
    w2 = 1 + (y - 1) / 4
    term1 = np.sin(np.pi * w1) ** 2
    term2 = ((w1 - 1) ** 2) * (1 + 10 * np.sin(np.pi * w1 + 1) ** 2)
    term3 = ((w2 - 1) ** 2) * (1 + np.sin(2 * np.pi * w2) ** 2)
    return term1 + term2 + term3



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-5, 5, -5, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Levy N.13)):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Levy N.13)):
Complete Graph: 0.048611
Hypercube Q8: 0.041088
Grid: 0.038429
Grid + Diagonals: 0.048611
Grid + Main Diagonals: 0.032191
QMOA Complete: 0.045371
QMOA Complete + local Diagonals: 0.045480
QMOA Complete + Principal Complete Diagonals: 0.013752
Grid + Complete Diagonals: 0.001794
Grid with Parallel Edges: 0.048611


In [24]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    a = np.array([3, 5, 2, 1, 7])
    b = np.array([5, 2, 1, 4, 9])
    c = np.array([1, 2, 5, 2, 3])
    m = len(a)

    total = 0
    for i in range(m):
        r2 = (x - a[i])**2 + (y - b[i])**2
        total += c[i] * np.exp(-r2 / np.pi) * np.cos(np.pi * r2)
    return total



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,0, 10, 0, 10, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Langerman):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Langerman):
Complete Graph: 0.429911
Hypercube Q8: 0.191590
Grid: 0.005710
Grid + Diagonals: 0.002774
Grid + Main Diagonals: 0.001360
QMOA Complete: 0.430155
QMOA Complete + local Diagonals: 0.332428
QMOA Complete + Principal Complete Diagonals: 0.073270
Grid + Complete Diagonals: 0.029379
Grid with Parallel Edges: 0.006966


In [9]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    return 2*x**2 - 1.05*(x**4) + (x**6)/6 + x*y + y**2



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-4.9, 5, -4.9, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Three Hump Camel):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Three Hump Camel):
Complete Graph: 0.050981
Hypercube Q8: 0.000424
Grid: 0.038429
Grid + Diagonals: 0.050981
Grid + Main Diagonals: 0.043096
QMOA Complete: 0.048478
QMOA Complete + local Diagonals: 0.050981
QMOA Complete + Principal Complete Diagonals: 0.017185
Grid + Complete Diagonals: 0.012122
Grid with Parallel Edges: 0.049205


In [26]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    return -np.cos(x) * np.cos(y) * np.exp(-((x - np.pi)**2 + (y - np.pi)**2))



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-7*np.pi, 8*np.pi,-7*np.pi, 8*np.pi, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Easom):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Easom):
Complete Graph: 0.323954
Hypercube Q8: 0.096264
Grid: 0.022307
Grid + Diagonals: 0.033122
Grid + Main Diagonals: 0.021084
QMOA Complete: 0.124023
QMOA Complete + local Diagonals: 0.119529
QMOA Complete + Principal Complete Diagonals: 0.149980
Grid + Complete Diagonals: 0.026243
Grid with Parallel Edges: 0.028503


## Rugged

In [10]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    return  20 + x**2 + y**2 - 10 * (np.cos(2 * np.pi * x) + np.cos(2 * np.pi * y))



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-4.9, 5, -4.9, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Rastrigin):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Rastrigin):
Complete Graph: 0.174496
Hypercube Q8: 0.036845
Grid: 0.038429
Grid + Diagonals: 0.076565
Grid + Main Diagonals: 0.043096
QMOA Complete: 0.189245
QMOA Complete + local Diagonals: 0.189244
QMOA Complete + Principal Complete Diagonals: 0.174200
Grid + Complete Diagonals: 0.000013
Grid with Parallel Edges: 0.075703


In [28]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y,m=10):
    term1 = np.sin(x) * (np.sin((x**2)/np.pi) ** (2 * m))
    term2 = np.sin(y) * (np.sin((2 * y**2)/np.pi) ** (2 * m))
    return -(term1 + term2)



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-5, 5, -5, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Michalewicz):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Michalewicz):
Complete Graph: 0.170874
Hypercube Q8: 0.223010
Grid: 0.036042
Grid + Diagonals: 0.073647
Grid + Main Diagonals: 0.039752
QMOA Complete: 0.297042
QMOA Complete + local Diagonals: 0.305744
QMOA Complete + Principal Complete Diagonals: 0.298172
Grid + Complete Diagonals: 0.071953
Grid with Parallel Edges: 0.049828


In [3]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    term1 = (x**2 + y**2) / 4000
    term2 = np.cos(x / np.sqrt(1)) * np.cos(y / np.sqrt(2))
    return term1 - term2 + 1



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-4.9, 5, -4.9, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Griewank):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Griewank):
Complete Graph: 0.007357
Hypercube Q8: 0.001622
Grid: 0.000010
Grid + Diagonals: 0.000038
Grid + Main Diagonals: 0.000084
QMOA Complete: 0.007357
QMOA Complete + local Diagonals: 0.007357
QMOA Complete + Principal Complete Diagonals: 0.007357
Grid + Complete Diagonals: 0.000593
Grid with Parallel Edges: 0.000038


In [4]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    return 0.26 * (x**2 + y**2) - 0.48 * x * y



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-4.9, 5, -4.9, 5, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Matyas):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Matyas):
Complete Graph: 0.002640
Hypercube Q8: 0.002638
Grid: 0.002640
Grid + Diagonals: 0.002640
Grid + Main Diagonals: 0.002640
QMOA Complete: 0.002640
QMOA Complete + local Diagonals: 0.002640
QMOA Complete + Principal Complete Diagonals: 0.002640
Grid + Complete Diagonals: 0.002640
Grid with Parallel Edges: 0.002640


In [5]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    r = np.sqrt(x**2 + y**2)
    return -np.abs(np.sin(x) * np.cos(y) * np.exp(np.abs(1 - r/np.pi)))



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-9.9, 10, -9.9, 10, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Holder Table):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Holder Table):
Complete Graph: 0.213922
Hypercube Q8: 0.018051
Grid: 0.001659
Grid + Diagonals: 0.003031
Grid + Main Diagonals: 0.000063
QMOA Complete: 0.213590
QMOA Complete + local Diagonals: 0.189497
QMOA Complete + Principal Complete Diagonals: 0.214774
Grid + Complete Diagonals: 0.009882
Grid with Parallel Edges: 0.003180


In [32]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    return np.sin(x + y) + (x - y)**2 - 1.5*x + 2.5*y + 1.0



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-1.5, 4.0, -3.0, 4.0, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(McCormick):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(McCormick):
Complete Graph: 0.014079
Hypercube Q8: 0.014079
Grid: 0.014079
Grid + Diagonals: 0.014079
Grid + Main Diagonals: 0.011074
QMOA Complete: 0.014079
QMOA Complete + local Diagonals: 0.014079
QMOA Complete + Principal Complete Diagonals: 0.014079
Grid + Complete Diagonals: 0.014079
Grid with Parallel Edges: 0.014079


In [6]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    x, y = np.asarray(x, dtype=float), np.asarray(y, dtype=float)
    term1 = np.sin(x) * np.sin(y)
    term2 = np.exp(np.abs(100 - (np.sqrt(x**2 + y**2) / np.pi)))
    return -0.0001 * (np.abs(term1 * term2) + 1)**0.1



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-9.9, 10, -9.9, 10, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Cross-in-tray):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Cross-in-tray):
Complete Graph: 0.013370
Hypercube Q8: 0.004471
Grid: 0.006566
Grid + Diagonals: 0.009820
Grid + Main Diagonals: 0.006821
QMOA Complete: 0.013370
QMOA Complete + local Diagonals: 0.013370
QMOA Complete + Principal Complete Diagonals: 0.009147
Grid + Complete Diagonals: 0.000316
Grid with Parallel Edges: 0.008950


In [7]:
import numpy as np
import networkx as nx
from scipy.linalg import eigh

# -------------------------
# Spectral gap calculator
# -------------------------
rows, cols = 16, 16
n = rows * cols 
def spectral_gap(H):
    evals = np.sort(eigh(H, eigvals_only=True))
    return evals[1] - evals[0]

def interpolate_gap(L, H_C, steps=200):
    s_vals = np.linspace(0, 1, steps)
    gaps = []
    for s in s_vals:
        Hs = (1-s) * L + s * H_C
        gaps.append(spectral_gap(Hs))
    return np.min(gaps)

# -------------------------
# Example Cost Hamiltonian
# (Diagonal encoding, replace with your real cost function!)
# -------------------------
def cost_fn(x, y):
    num = np.sin(x**2 - y**2)**2 - 0.5
    den = (1 + 0.001*(x**2 + y**2))**2
    return 0.5 + num / den



def build_cost_hamiltonian(cost_func, x_min, x_max, y_min, y_max, rows, cols):
    dx = (x_max - x_min) / (rows - 1)
    dy = (y_max - y_min) / (cols - 1)

    costs = []
    for i in range(rows):
        for j in range(cols):
            x = x_min + i * dx
            y = y_min + j * dy
            costs.append(cost_func(x, y))
    return np.diag(costs)

def build_grid(rows, cols):
    return nx.convert_node_labels_to_integers(nx.grid_2d_graph(rows, cols))

def build_grid_with_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))
            G.add_edge((i + 1, j), (i, j + 1))
    return nx.convert_node_labels_to_integers(G)

def build_grid_with_main_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)
    for i in range(min(rows - 1, cols - 1)):
        G.add_edge((i, i), (i + 1, i + 1))
        G.add_edge((i, cols - 1 - i), (i + 1, cols - 2 - i))
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_complete(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    return nx.convert_node_labels_to_integers(G)

def build_qmoa_with_diagonals(rows, cols):
    G = nx.Graph()
    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))
    # All-to-all in rows
    for r in range(rows):
        row_nodes = [(r, c) for c in range(cols)]
        for i in range(len(row_nodes)):
            for j in range(i + 1, len(row_nodes)):
                G.add_edge(row_nodes[i], row_nodes[j])
    # All-to-all in columns
    for c in range(cols):
        col_nodes = [(r, c) for r in range(rows)]
        for i in range(len(col_nodes)):
            for j in range(i + 1, len(col_nodes)):
                G.add_edge(col_nodes[i], col_nodes[j])
    # Add diagonals
    for i in range(rows - 1):
        for j in range(cols - 1):
            G.add_edge((i, j), (i + 1, j + 1))  # Forward diagonal
            G.add_edge((i + 1, j), (i, j + 1))  # Backward diagonal

    # Relabel nodes to 0...n-1 to match H_C
    return nx.convert_node_labels_to_integers(G, ordering="sorted")
def build_qmoa_with_principal_complete_diagonals(rows, cols):
    G = nx.Graph()

    for r in range(rows):
        for c in range(cols):
            G.add_node((r, c))

    # Row-wise complete connections
    for r in range(rows):
        row = [(r, c) for c in range(cols)]
        for i in range(len(row)):
            for j in range(i + 1, len(row)):
                G.add_edge(row[i], row[j])

    # Column-wise complete connections
    for c in range(cols):
        col = [(r, c) for r in range(rows)]
        for i in range(len(col)):
            for j in range(i + 1, len(col)):
                G.add_edge(col[i], col[j])

    # Principal main diagonal (↘)
    main_diag = [(i, i) for i in range(min(rows, cols))]
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Principal anti-diagonal (↙)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_grid_with_complete_diagonals(rows, cols):
    G = nx.grid_2d_graph(rows, cols)

    # Collect nodes along the main diagonal (top-left to bottom-right)
    main_diag = [(i, i) for i in range(min(rows, cols))]

    # Collect nodes along the anti-diagonal (top-right to bottom-left)
    anti_diag = [(i, cols - 1 - i) for i in range(min(rows, cols))]

    # Add complete edges among all nodes on main diagonal
    for i in range(len(main_diag)):
        for j in range(i + 1, len(main_diag)):
            G.add_edge(main_diag[i], main_diag[j])

    # Add complete edges among all nodes on anti-diagonal
    for i in range(len(anti_diag)):
        for j in range(i + 1, len(anti_diag)):
            G.add_edge(anti_diag[i], anti_diag[j])

    return nx.convert_node_labels_to_integers(G)

def build_multigraph_grid(rows, cols, edge_multiplicity=2):
    G = nx.MultiGraph()
    for i in range(rows):
        for j in range(cols):
            if j < cols - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i, j + 1))
            if i < rows - 1:
                for _ in range(edge_multiplicity):
                    G.add_edge((i, j), (i + 1, j))
    return G

def multigraph_to_weighted_graph(G_multi):
    G_weighted = nx.Graph()
    for u, v in G_multi.edges():
        if G_weighted.has_edge(u, v):
            G_weighted[u][v]['weight'] += 1
        else:
            G_weighted.add_edge(u, v, weight=1)
    return nx.convert_node_labels_to_integers(G_weighted)

# -------------------------
# Graph Laplacians
# -------------------------
def build_laplacians(n):
    graphs = {
    "Complete Graph": nx.complete_graph(n),
    "Hypercube Q8": nx.convert_node_labels_to_integers(nx.hypercube_graph(8)),
    "Grid": build_grid(rows, cols),
    "Grid + Diagonals": build_grid_with_diagonals(rows, cols),
    "Grid + Main Diagonals": build_grid_with_main_diagonals(rows, cols),
    "QMOA Complete": build_qmoa_complete(rows, cols),
    "QMOA Complete + local Diagonals": build_qmoa_with_diagonals(rows, cols),
    "QMOA Complete + Principal Complete Diagonals": build_qmoa_with_principal_complete_diagonals(rows, cols),
    "Grid + Complete Diagonals" : build_grid_with_complete_diagonals(rows, cols) }
    # Add weighted version of multiedge grid
    G_multi_raw = build_multigraph_grid(rows, cols, edge_multiplicity=2)
    G_multi_weighted = multigraph_to_weighted_graph(G_multi_raw)
    graphs["Grid with Parallel Edges"] = G_multi_weighted

    Ls = {}
    for name, G in graphs.items():
        if G is not None:
            Ls[name] = nx.laplacian_matrix(G).toarray()
    return Ls

# -------------------------
# Run comparison
# -------------------------
def compare_gaps():
    H_C = build_cost_hamiltonian(cost_fn,-99.9, 100, -99.9, 100, rows, cols)
    Ls = build_laplacians(n)
    results = {}
    for name, L in Ls.items():
        Δmin = interpolate_gap(L, H_C)
        results[name] = Δmin
    return results


# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
    results = compare_gaps()
    print("Minimum Spectral Gaps(Schaffer function N. 2):")
    for graph, gap in results.items():
        print(f"{graph}: {gap:.6f}")



Minimum Spectral Gaps(Schaffer function N. 2):
Complete Graph: 0.002065
Hypercube Q8: 0.000312
Grid: 0.002065
Grid + Diagonals: 0.002065
Grid + Main Diagonals: 0.002065
QMOA Complete: 0.002065
QMOA Complete + local Diagonals: 0.002065
QMOA Complete + Principal Complete Diagonals: 0.002065
Grid + Complete Diagonals: 0.001078
Grid with Parallel Edges: 0.002065
